In [ ]:
import pandas as pd
import numpy as np
import pickle

In [ ]:
sel_col=['user_id','video_id','time_ms','is_click','is_like','is_hate','play_time_ms']
df1_part1 = pd.read_csv("data/log_standard_4_08_to_4_21_27k_part1.csv", nrows=100000000, usecols=sel_col)

In [ ]:
df_click=df1_part1[df1_part1['is_click']==1]

In [ ]:
df=df_click

In [ ]:
hate_rows = df[df['is_hate'] == 1][['user_id', 'video_id']]

In [ ]:
# is_hate 
result_df = df[(df['user_id'].isin(hate_rows['user_id'])) | (df['video_id'].isin(hate_rows['video_id']))]

In [ ]:
id_counts = result_df['video_id'].value_counts()
ids_to_remove = id_counts[id_counts < 5].index
df2 = result_df[~result_df['video_id'].isin(ids_to_remove) | result_df['video_id'].isin(hate_rows['video_id'])]

In [ ]:
id_counts = df2['user_id'].value_counts()
ids_to_remove = id_counts[id_counts < 5].index
df_filtered = df2[~df2['user_id'].isin(ids_to_remove)| df2['user_id'].isin(hate_rows['user_id'])]

In [ ]:
id_counts = df_filtered['video_id'].value_counts()
ids_to_remove = id_counts[id_counts < 5].index
df2 = df_filtered[~df_filtered['video_id'].isin(ids_to_remove) | df_filtered['video_id'].isin(hate_rows['video_id'])]

In [ ]:
df=df2

In [ ]:
df['satis'] = 0.5  
df.loc[df['is_like'] == 1, 'satis'] = 1  
df.loc[df['is_hate'] == 1, 'satis'] = 0  

In [ ]:
df = df.drop(df[(df['is_hate'] == 1) & (df['is_like'] == 1)].index)

In [ ]:
df = df.drop_duplicates() 

In [ ]:
id_counts = df['user_id'].value_counts()
ids_to_remove = id_counts[id_counts < 5].index
df_filtered = df[~df['user_id'].isin(ids_to_remove)| df['user_id'].isin(hate_rows['user_id'])]

In [ ]:
id_counts = df_filtered['video_id'].value_counts()
ids_to_remove = id_counts[id_counts < 5].index
df = df_filtered[~df_filtered['video_id'].isin(ids_to_remove) | df_filtered['video_id'].isin(hate_rows['video_id'])]

In [ ]:
id_counts = df['user_id'].value_counts()
ids_to_remove = id_counts[id_counts < 5].index
df_filtered = df[~df['user_id'].isin(ids_to_remove)| df['user_id'].isin(hate_rows['user_id'])]

In [ ]:
data_click=df_filtered

In [ ]:
train_data_list = []
validation_data_list = []
test_data_list = []
u_id_less=[]

for _, group_data in data_click.groupby('user_id'):
    if len(group_data) >= 5:
        test_data_list.append(group_data.iloc[-1])
        validation_data_list.append(group_data.iloc[-2])
        train_data_list.append(group_data.iloc[:-2])
    else:
        u_id_less.append(group_data)
        print('error')

In [ ]:
train_data_list=pd.concat(train_data_list)

In [ ]:
train_data = pd.DataFrame(train_data_list)
validation_data = pd.DataFrame(validation_data_list)
test_data = pd.DataFrame(test_data_list)

In [ ]:
train_data.sort_values(by=['user_id','time_ms'], inplace=True)

In [ ]:
data_sel=data_click[data_click['user_id'].isin(train_data['user_id'])]

In [ ]:
df_sel=data_sel
click_h = df_sel[df_sel['is_click'] == 1]

In [ ]:
grouped_df = click_h.groupby('user_id').apply(lambda x: x.sort_values('time_ms'))
result_df = grouped_df[['video_id','play_time_ms','time_ms']]
unique_values_a = result_df.index.get_level_values(0).unique()

In [ ]:
user_his={}
for user_id in unique_values_a:
    user_data = result_df.loc[user_id]
    user_his[user_id]={}    
    user_his[user_id]['click_his']=user_data['video_id'].tolist()
    user_his[user_id]['play_time_ms']=user_data['play_time_ms'].tolist()
    user_his[user_id]['click_time_ms']=user_data['time_ms'].tolist()        

In [ ]:
# satis his
satis_h = df_sel[df_sel['satis'] == 1]
grouped_df = satis_h.groupby('user_id').apply(lambda x: x.sort_values('time_ms'))
result_df = grouped_df[['video_id','time_ms']]
unique_values_a = result_df.index.get_level_values(0).unique()
satis_his={}
for user_id in unique_values_a:
    user_data = result_df.loc[user_id]
    satis_his[user_id]={}
    satis_his[user_id]['satis_his']=user_data['video_id'].tolist()
    satis_his[user_id]['satis_time_ms']=user_data['time_ms'].tolist()    

In [ ]:
for key in user_his.keys():
    if key in satis_his:
        user_his[key].update(satis_his[key])
    else:
        satis_his[key]={}
        satis_his[key]['satis_his']=[]
        satis_his[key]['satis_time_ms']=[]
        user_his[key].update(satis_his[key])

In [ ]:
# dissatis his
dissatis_h = df_sel[df_sel['satis'] == 0]
grouped_df = dissatis_h.groupby('user_id').apply(lambda x: x.sort_values('time_ms'))
result_df = grouped_df[['video_id','time_ms']]
unique_values_a = result_df.index.get_level_values(0).unique()
dissatis_his={}
for user_id in unique_values_a:
    user_data = result_df.loc[user_id]
    dissatis_his[user_id]={}
    dissatis_his[user_id]['dissatis_his']=user_data['video_id'].tolist()
    dissatis_his[user_id]['dissatis_time_ms']=user_data['time_ms'].tolist()

In [ ]:
for key in user_his.keys():
    if key in dissatis_his:
        user_his[key].update(dissatis_his[key])
    else:
        dissatis_his[key]={}
        dissatis_his[key]['dissatis_his']=[]
        dissatis_his[key]['dissatis_time_ms']=[]
        user_his[key].update(dissatis_his[key])

In [ ]:
# for user features
user_features = pd.read_csv("data/user_features_27k.csv")

In [ ]:
print(user_features['onehot_feat0'].max())
print(user_features['onehot_feat1'].max())
print(user_features['onehot_feat2'].max())
print(user_features['onehot_feat3'].max())
print(user_features['onehot_feat5'].max())
print(user_features['onehot_feat6'].max())
print(user_features['onehot_feat7'].max())
print(user_features['onehot_feat8'].max())
print(user_features['onehot_feat9'].max())
print(user_features['onehot_feat10'].max())
print(user_features['onehot_feat11'].max())

In [ ]:
user_features = user_features.dropna(axis=1)

In [ ]:
sel_cols=['user_id','onehot_feat2','onehot_feat5','onehot_feat7','onehot_feat8']

In [ ]:
user_features=user_features[sel_cols]
user_features=user_features[user_features['user_id'].isin(train_data['user_id'])]

In [ ]:
# add id
user_features['id']=range(len(user_features))

In [ ]:
# to dict
user_vocab = user_features.set_index('user_id').to_dict(orient='index')

In [ ]:
for key in user_vocab.keys():
    if key in user_his:
        user_vocab[key].update(user_his[key])       

In [ ]:
with open('kuairand_user_vocab.pickle', 'wb') as pickle_file:
    pickle.dump(user_vocab, pickle_file)

In [ ]:
df=df_sel

In [ ]:
df.sort_values(by=['user_id','time_ms'], inplace=True)

In [ ]:
df['count_click'] = df.groupby('user_id')['is_click'].apply(lambda x: (x == 1) ).groupby(df['user_id']).cumsum()
df['count_click'] -= (df['is_click'] == 1).astype(int)

In [ ]:
df['cnt_satis'] = df.groupby('user_id')['is_like'].apply(lambda x: (x == 1) ).groupby(df['user_id']).cumsum()
df['cnt_satis'] -= (df['is_like'] == 1).astype(int)

In [ ]:
df['cnt_dissatis'] = df.groupby('user_id')['is_hate'].apply(lambda x: (x == 1) ).groupby(df['user_id']).cumsum()
df['cnt_dissatis'] -= (df['is_hate'] == 1).astype(int)

In [ ]:
data_click=df

In [ ]:
train_data_list = []
validation_data_list = []
test_data_list = []
u_id_less=[]

for _, group_data in data_click.groupby('user_id'):
    if len(group_data) >= 5:
        test_data_list.append(group_data.iloc[-1])
        validation_data_list.append(group_data.iloc[-2])
        train_data_list.append(group_data.iloc[:-2])
    else:
        u_id_less.append(group_data)
        print('error')

In [ ]:
train_data_list=pd.concat(train_data_list)

In [ ]:
train_data = pd.DataFrame(train_data_list)
validation_data = pd.DataFrame(validation_data_list)
test_data = pd.DataFrame(test_data_list)

In [ ]:
train_data.sort_values(by=['user_id','time_ms'], inplace=True)

In [ ]:
u_id_list = train_data['user_id'].unique()

In [ ]:
i_id_list = data_click['video_id'].unique()

In [ ]:
# val
negative_items_dict_val2 = {}
i=0
for u_id in u_id_list:
    if i%500==0:print(i)
    i+=1
    click_history = user_vocab[u_id]['click_his'][:-2]
    sample_set_np = np.array(i_id_list)
    mask = np.isin(sample_set_np, click_history)
    filtered_i_id_list=sample_set_np[~mask]
    negative_samples = np.random.choice(filtered_i_id_list, size=99)
    negative_items_dict_val2[u_id] = negative_samples.tolist()

In [ ]:
validation_negative_items = []
for u_id, neg_items in negative_items_dict_val2.items():
    validation_negative_items.extend(neg_items)

In [ ]:
# test
negative_items_dict_test2 = {}
i=0
for u_id in u_id_list:
    if i%500==0:print(i)
    i+=1
    click_history = user_vocab[u_id]['click_his'][:-1]
    sample_set_np = np.array(i_id_list)
    mask = np.isin(sample_set_np, click_history)
    filtered_i_id_list=sample_set_np[~mask]
    negative_samples = np.random.choice(filtered_i_id_list, size=99)
    negative_items_dict_test2[u_id] = negative_samples.tolist()

In [ ]:
test_negative_items = []
for u_id, neg_items in negative_items_dict_test2.items():
    test_negative_items.extend(neg_items)

In [ ]:
validation_data_item = pd.DataFrame({'user_id': u_id_list.repeat(99), 'video_id': validation_negative_items})
test_data_item = pd.DataFrame({'user_id': u_id_list.repeat(99), 'video_id': test_negative_items})

In [ ]:
validation_data = pd.concat([validation_data, validation_data_item], ignore_index=True)
test_data = pd.concat([test_data, test_data_item], ignore_index=True)

In [ ]:
validation_data.sort_values(by=['user_id','satis'], inplace=True) 
validation_data['time_ms'] = validation_data['time_ms'].fillna(method='ffill')
validation_data['is_click'] = validation_data['is_click'].fillna(0)
validation_data['cnt_dissatis'] = validation_data['cnt_dissatis'].fillna(method='ffill')
validation_data['play_time_ms'] = validation_data['play_time_ms'].fillna(method='ffill')
validation_data['cnt_satis'] = validation_data['cnt_satis'].fillna(method='ffill')
validation_data['count_click'] = validation_data['count_click'].fillna(method='ffill')
validation_data['satis'] = validation_data['satis'].fillna(0.5)

In [ ]:
test_data.sort_values(by=['user_id','satis'], inplace=True)
test_data['time_ms'] = test_data['time_ms'].fillna(method='ffill')
test_data['cnt_dissatis'] = test_data['cnt_dissatis'].fillna(method='ffill')
test_data['play_time_ms'] = test_data['play_time_ms'].fillna(method='ffill')
test_data['cnt_satis'] = test_data['cnt_satis'].fillna(method='ffill')
test_data['count_click'] = test_data['count_click'].fillna(method='ffill')
test_data['is_click'] = test_data['is_click'].fillna(0)
test_data['satis'] = test_data['satis'].fillna(0.5)

In [ ]:
train_data=train_data[train_data['count_click']>0]

In [ ]:
# item features
video_features_basic = pd.read_csv("data/video_features_basic_27k.csv")

In [ ]:
item_vocab=video_features_basic[video_features_basic['video_id'].isin(data_click['video_id'])]

In [ ]:
active_values = item_vocab['upload_type'].unique()
active_encoding_dict = {value: idx for idx, value in enumerate(active_values)}
item_vocab['upload_type']=item_vocab['upload_type'].map(active_encoding_dict)

In [ ]:
active_values = item_vocab['music_type'].unique()
active_encoding_dict = {value: idx for idx, value in enumerate(active_values)}
item_vocab['music_type']=item_vocab['music_type'].map(active_encoding_dict)

In [ ]:
# add id
item_vocab['id']=range(len(item_vocab))
item_vocab['id']+=1

In [ ]:
sel_cols=['video_id','id','upload_type','music_type','tag']
item_vocab=item_vocab[sel_cols]

In [ ]:
print(item_vocab['upload_type'].max())
print(item_vocab['music_type'].max())

In [ ]:
# to dict
item_vocab = item_vocab.set_index('video_id').to_dict(orient='index')

In [ ]:
with open('kuairand_item_vocab.pickle', 'wb') as pickle_file:
    pickle.dump(item_vocab, pickle_file)

In [ ]:
df=train_data

In [ ]:
valid_users = df[df['is_like'] == 1 | (df['is_hate'] == 1)]['user_id']
filtered_df = df[df['user_id'].isin(valid_users)]

In [ ]:
train_data=filtered_df

In [ ]:
validation_data=validation_data[validation_data['user_id'].isin(train_data['user_id'])]
test_data=test_data[test_data['user_id'].isin(train_data['user_id'])]

In [ ]:
df2=train_data

In [ ]:
hate_rows = df2[df2['is_hate'] == 1][['user_id', 'video_id']]

In [ ]:
id_counts = df2['user_id'].value_counts()
ids_to_remove = id_counts[id_counts < 3].index
df_filtered = df2[~df2['user_id'].isin(ids_to_remove)| df2['user_id'].isin(hate_rows['user_id'])]

In [ ]:
id_counts = df_filtered['video_id'].value_counts()
ids_to_remove = id_counts[id_counts < 5].index
df2 = df_filtered[~df_filtered['video_id'].isin(ids_to_remove) | df_filtered['video_id'].isin(hate_rows['video_id'])]

In [ ]:
id_counts = df2['user_id'].value_counts()
ids_to_remove = id_counts[id_counts < 3].index
df_filtered = df2[~df2['user_id'].isin(ids_to_remove)| df2['user_id'].isin(hate_rows['user_id'])]

In [ ]:
id_counts = df_filtered['video_id'].value_counts()
ids_to_remove = id_counts[id_counts < 5].index
df2 = df_filtered[~df_filtered['video_id'].isin(ids_to_remove) | df_filtered['video_id'].isin(hate_rows['video_id'])]

In [ ]:
validation_data=validation_data[validation_data['user_id'].isin(train_data['user_id'])]
test_data=test_data[test_data['user_id'].isin(train_data['user_id'])]

In [ ]:
train_data=df2

In [ ]:
train_data.to_csv('kuairand_train.tsv',sep='\t',index=False)
validation_data.to_csv('kuairand_val.tsv',sep='\t',index=False)
test_data.to_csv('kuairand_test.tsv',sep='\t',index=False)

In [ ]:
u_id_list = train_data['user_id'].unique()

In [ ]:
for key in list(user_vocab.keys()):
    if key not in u_id_list:
        user_vocab.pop(key)

In [ ]:
id_cnt=0
for key, value in user_vocab.items():
    value['id'] = id_cnt  
    id_cnt+=1

In [ ]:
with open('kuairand_user_vocab.pickle', 'wb') as pickle_file:
    pickle.dump(user_vocab, pickle_file)

In [ ]:
# for satis tower
df=train_data

In [ ]:
like_users = df[df['is_like'] == 1]['user_id'].unique()
hate_users = df[df['is_hate'] == 1]['user_id'].unique()
both_like_and_hate_users = set(like_users) & set(hate_users)
df_both = df[df['user_id'].isin(both_like_and_hate_users)]

In [ ]:
df_reward=df_both[df_both['satis']!=0.5]

In [ ]:
df_reward.to_csv('kuairand_satis.tsv',sep='\t',index=False)

In [ ]:
train_data.loc[~train_data['user_id'].isin(df_reward['user_id']), 'satis'] = 0.5

In [ ]:
train_data.to_csv('kuairand_train.tsv',sep='\t',index=False)